### file 읽어오기

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
from langchain_community.document_loaders import TextLoader
path = r'data/medicine.txt'

# 1. loader 객체 생성
loader = TextLoader(path, encoding='utf-8')
# 2. loader를 이용해서 파일 읽어옴
docs = loader.load()

In [13]:
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.globals import set_llm_cache
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

set_llm_cache(SQLiteCache(".cache_prompt.sqlite"))

COLLECTION_NAME = "medicine_docs"
PERSIST_DIRECTORY = "vector_store/chroma/medicine_db7"
EMBEDDING_MODEL_NAME = 'text-embedding-3-small'
from langchain_openai import OpenAIEmbeddings

# Text Loading
loader = TextLoader(path, encoding='utf-8')

# 문서 load and split
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

docs = loader.load_and_split(splitter)

embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs = {
        'k':5,
        'fetch_k':10
    }
)



In [14]:
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]
    
    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id],
        k=2,
        return_messages=True,
        message_key="history"
    )
    message_list = memory.load_memory_variables({})['history']
    store[session_id] = InMemoryChatMessageHistory(messages=message_list)
    return store[session_id]

In [27]:
from geopy.distance import geodesic
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import json


def read_file() :
    # CSV 파일 경로 설정
    pharmacy_data_path = r"data/pharmacy.csv"

    # CSV 파일 읽기
    try:
        return pd.read_csv(pharmacy_data_path, encoding='utf-8')
    except Exception as e:
        return None

pharmacy_data = read_file()

# 사용자 IP를 통해 위치 가져오기
def get_user_location_by_ip():
    try:
        load_dotenv()
        google_api_key = os.environ.get('GOOGLE_API_KEY')
        url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={google_api_key}'

        response = json.loads(requests.post(url).text)
        
        location = response['location']
        lat = round(location['lat'], 2)
        lng = round(location['lng'], 2)
        lat_lng = (lat, lng)
        
        return lat_lng
    except Exception as e:
        print("IP 기반 위치를 가져오는 데 실패했습니다.", str(e))
        return None

# 시간 변환 함수
def convert_time_format(time):
    """900 → 09:00 형식으로 변환"""
    try:
        time = str(int(time))  # 정수로 변환 후 문자열로 처리
        if len(time) == 3:  # 3자리인 경우 (예: 900)
            time = f"0{time[0]}:{time[1:]}"
        elif len(time) == 4:  # 4자리인 경우 (예: 1830)
            time = f"{time[:2]}:{time[2:]}"
        return time
    except ValueError:
        return "정보 없음"

# 운영시간 변환 함수
def format_operating_hours(row, day):
    """운영시간을 시작~종료 형식으로 변환"""
    start_time = row.get(f'진료시간({day})S', '정보 없음')  # 시작 시간
    close_time = row.get(f'진료시간({day})C', '정보 없음')  # 종료 시간
    
    if start_time != '정보 없음' and close_time != '정보 없음':
        return f"{convert_time_format(start_time)}~{convert_time_format(close_time)}"
    else:
        return "정보 없음"

# 근처 약국 추천 함수
def recommend_nearby_pharmacies(user_location, max_distance_km=1, num_pharmacies=2):
    if pharmacy_data is None:
        print("약국 데이터가 없습니다. CSV 파일을 확인하세요.")
        return []

    recommended_pharmacies = []

    for _, row in pharmacy_data.iterrows():
        try:
            # 위도와 경도 열 이름 확인
            pharmacy_location = (float(row['병원위도']), float(row['병원경도']))
            distance = geodesic(user_location, pharmacy_location).km

            if distance <= max_distance_km:
                recommended_pharmacies.append({
                    "name": row['약국명'],
                    "address": row['주소'],
                    "phone": row.get('대표전화1', '정보 없음'),
                    "distance": round(distance, 2),
                    "hours": {
                        "weekday": format_operating_hours(row, '월요일'),
                        "saturday": format_operating_hours(row, '토요일'),
                        "sunday": format_operating_hours(row, '일요일')
                    }
                })
        except Exception as e:
            print("약국 데이터 처리 중 오류:", str(e))
            continue

    # 거리순으로 정렬 후 상위 n개 선택
    recommended_pharmacies = sorted(recommended_pharmacies, key=lambda x: x['distance'])[:num_pharmacies]

    return recommended_pharmacies


def recommend_pharmacy() :
    # 사용자 위치 가져오기 및 약국 추천 실행
    if pharmacy_data is not None:
        user_location = get_user_location_by_ip()
        text = ''
        if user_location:
            nearby_pharmacies = recommend_nearby_pharmacies(user_location)

            # 결과 출력
            if nearby_pharmacies:
                for pharmacy in nearby_pharmacies:
                    text += f"{pharmacy['name']} {pharmacy['address']} {pharmacy['phone']} {pharmacy['distance']} km 평일: {pharmacy['hours']['weekday']} 토요일: {pharmacy['hours']['saturday']} 일요일: {pharmacy['hours']['sunday']}\n"
            else:
                text = "근처에 추천할 약국이 없습니다."
        else:
            text = "사용자 위치를 가져오는 데 실패했습니다."

        summary_prompt = PromptTemplate(
            template="근처 약국에 대한 내용이야. 다음 내용을 요약해줘.\n[요약할 내용]\n{content}"
        )
        summary_model = ChatOpenAI(model="gpt-4o-mini")
        summary_chain = summary_prompt | summary_model | StrOutputParser()
        summary_result = summary_chain.invoke({"content":text})
        return summary_result
    else:
        print("약국 데이터를 로드하지 못했습니다. 파일 경로와 내용을 확인하세요.")

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain.memory import ConversationBufferWindowMemory
from operator import itemgetter
from langchain_openai import ChatOpenAI

# Prompt Template 생성
messages = [
            ("ai", """
            너는 유능한 약사야. context 안에 있는 내용에 한 해서
            어떤 효과가 있는지 알려주고 사용자가 입력한 증상에 맞는 약을 5개만 선택해서 알려줘.
            복용이 간단하고 부작용이 적은 약을 우선적으로 추천해서 알려줘.
            약을 복용하기 전에 반드시 알아야 할 사항은 꼭 알려줘.
            증상이 악화될 경우 대처방법에 대해서 알려줘.
            약에 따른 복용법에 대해서도 간단히 언급해줘.
            주의사항 요약해서 알려줘. 
            위의 내용을 표로 정리해서 알려줘.
            사용자가 말한 증상을 효능 부분에서 찾아서 그에 맞는 약을 알려줘.

            그리고 약국에 대한 정보를 물어보면 'recommend_pharmacy'라고만 트리거 줘
            
            {context}")"""),
            MessagesPlaceholder("history"), 
            ("human", "{question}")
    ]

prompt_template = ChatPromptTemplate(messages)

model = ChatOpenAI(model='gpt-4o-mini')

parser = StrOutputParser()

def get_context(input_data:dict):
    return retriever.invoke(input_data['question'])

runnable = {'context':RunnableLambda(get_context), 'question':itemgetter("question"), 'history':itemgetter('history')} | prompt_template | model | parser

chain = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

config = {"configurable": {"session_id":"id-1"}}
data = chain.invoke({"question":"약국 알려줘"}, config)

if data == 'recommend_pharmacy' :
    data = recommend_pharmacy()

print(data)


가람약국은 서울 금천구 가산디지털1로 5, 대륭테크노타운 20차 107호에 위치하며, 전화번호는 02-2225-0048입니다. 운영시간은 평일 08:00~20:00, 토요일 09:00~14:30, 일요일 09:00~22:00입니다. 거리 0.7km입니다.

레아약국은 서울 금천구 가산디지털1로 131, 117-1호 (BYC하이시티)에 위치하고, 전화번호는 02-6947-7772입니다. 운영시간은 평일 08:30~20:00, 토요일 08:30~14:00, 일요일 10:00~19:00이며, 거리 0.75km입니다.


In [ ]:
data = chain.invoke({"question":"첫번째 약에 대해서 더 설명해줘"}, config)
print(data)

/var/folders/8y/jsfncy6n39v2jylvdpm61nmh0000gn/T/ipykernel_35880/1752197439.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


속콜펜정에 대한 자세한 설명은 다음과 같습니다.

### 속콜펜정

**효능 및 효과:**
- 속콜펜정은 두통, 치통, 발치 후 동통, 인후통, 귀의 통증 등 다양한 통증을 완화하는 데 사용됩니다. 특히, 일반적인 두통 및 긴장성 두통에 효과적입니다.

**주 성분:**
- 주성분은 아세트아미노펜과 같은 진통제입니다. 이 성분은 뇌의 통증 신호를 차단하고, 열을 낮추는 작용을 합니다.

**복용법:**
- 성인(만 15세 이상): 1회 1정, 1일 3회 복용하며, 복용 간격은 최소 4시간 이상 두어야 합니다.
- 필요에 따라 의사의 지시에 따라 복용량을 조정할 수 있습니다.

**복용 시 주의사항:**
- **과민증:** 아세트아미노펜에 과민증이 있는 환자는 복용을 피해야 합니다.
- **간 질환:** 간 장애가 있는 환자, 특히 알코올을 자주 섭취하는 사람은 사용 시 주의해야 합니다.
- **임신 및 수유:** 임신 중이거나 수유 중인 여성은 의사와 상담 후 사용해야 합니다.
- **장기 복용:** 장기간 복용하지 말고, 5~6일 이상 복용 후에도 증상이 개선되지 않으면 의사와 상담해야 합니다.

**부작용:**
- 일반적으로 부작용이 적지만, 드물게 간 손상, 피부 발진, 알레르기 반응 등이 발생할 수 있습니다. 이러한 증상이 나타나면 즉시 복용을 중단하고 의사와 상담해야 합니다.

**보관 방법:**
- 어린이의 손이 닿지 않는 곳에 보관하고, 습기와 빛을 피해 실온에서 보관해야 합니다.

속콜펜정은 일반적으로 안전성이 높은 약물로, 두통 완화에 효과적입니다. 그러나 복용 전에 자신의 건강 상태와 병력에 대해 충분히 고려하고, 필요할 경우 전문의와 상담하는 것이 좋습니다.
